In [ ]:
import os
import boto3
from sagemaker import image_uris

In [ ]:
# AWSのリージョンとアカウントIDを取得
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

# ECRのリポジトリ名とイメージのタグを設定
repository_name = 'whisper-transcribe'
image_tag = 'GPU'

# DockerイメージのURIを作成
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{repository_name}:{image_tag}'

In [ ]:
# 作業ディレクトリに移動
os.chdir('/home/ec2-user/SageMaker/')

In [ ]:
# SageMaker公式のPyTorchの推論用Dockerイメージ名を取得
image_uris.retrieve(framework='pytorch',region='ap-northeast-1',version='2.2.0',py_version='py310',image_scope='inference', instance_type='ml.g4dn.xlarge')

In [ ]:
# AWS公式のDockerイメージをPull
os.system('aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com')
os.system('docker pull 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-inference:2.2.0-gpu-py310')

In [ ]:
# Dockerイメージをビルド
os.system(f'docker build -t {repository_name}:{image_tag} .')

In [ ]:
# ECRにログイン
os.system(f'$(aws ecr get-login --region {region} --no-include-email)')

# ECRのリポジトリを作成（すでに存在する場合はスキップ）
os.system(f'aws ecr create-repository --repository-name {repository_name}')

# DockerイメージをECRにプッシュ
os.system(f'docker tag {repository_name}:{image_tag} {image_uri}')
os.system(f'docker push {image_uri}')